In [7]:
!kaggle competitions download - c house-prices-advanced-regression-techniques
!unzip house-prices-advanced-regression-techniques.zip - d dataset


house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: dataset/data_description.txt  
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


In [138]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split, WeightedRandomSampler, Subset
import pandas as pd
import os
import re
from collections import Counter
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import gc
from tqdm import tqdm

In [320]:
current_path = os.getcwd()
train_path = os.path.join(current_path, "./dataset/train.csv")
test_path = os.path.join(current_path, "./dataset/test.csv")
submission_path = os.path.join(current_path, "./dataset/sample_submission.csv")


In [322]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
submission_df = pd.read_csv(submission_path)
len(train_df)


1460

In [30]:
from enum import Enum


class Filter_type(Enum):
    HIGH = 0
    MID = 1
    LOW = 2


def transform_df2input(df, mode: Filter_type, nor_mean=0, nor_std=1):
    if mode == Filter_type.HIGH:
        feature_list = ['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                        'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
                        'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
                        'WoodDeckSF', 'OpenPorchSF', 'SalePrice']
    elif mode == Filter_type.MID:
        feature_list = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF',
                        '1stFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces',
                        'GarageYrBlt', 'GarageCars', 'SalePrice']

    elif mode == Filter_type.LOW:
        feature_list = ['OverallQual', 'TotalBsmtSF', 'GrLivArea', 'GarageCars',
                        'SalePrice']

    df_selected = df[feature_list]]
    numpy_data = df_selected.values

    mean = np.nanmean(numpy_data, axis=0)
    std = np.nanstd(numpy_data, axis=0)

    # 정규화 수행
    normalized_array = (((numpy_data - mean) / (std) * nor_std)) + nor_mean
    normalized_array = np.nan_to_num(normalized_array)
    return numpy_data, normalized_array


data, nor_data = transform_df2input(train_df, Filter_type.HIGH)
nor_data.shape


1460
1460


(1460, 18)

In [313]:
class Dataset_type(Enum):
    TRAIN = 0
    TEST = 1


class House_dataset(Dataset):
    def __init__(self, df_path, dataset_type: Dataset_type, mode: Filter_type, nor_mean=0, nor_std=1):
        self.df_path = df_path
        self.df = pd.read_csv(df_path)
        self.nor_mean = nor_mean
        self.nor_std = nor_std
        self.dataset_type = dataset_type
        self.mode = mode
        self.input = self._transform_df2input(
            self.df, self.mode, self.nor_mean, self.nor_std)
        self.trg_mean, self.trg_std = self._trg_mean_std(self.input[1])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        src = self.input[0][idx] if self.dataset_type == Dataset_type.TRAIN else self.input[idx]
        if self.dataset_type == Dataset_type.TRAIN:
            trg = self.input[1][idx]
            trg = (((trg - self.trg_mean) / (self.trg_std)
                   * self.nor_std)) + self.nor_mean
            trg = np.nan_to_num(trg)
            return src, trg
        else:
            return src

    def _trg_mean_std(self, trg_list):
        mean = np.nanmean(trg_list, axis=0)
        std = np.nanstd(trg_list, axis=0)
        return mean, std

    def _transform_df2input(self, df, mode: Filter_type, dataset_type: Dataset_type, nor_mean=0, nor_std=1):
        if mode == Filter_type.HIGH:
            feature_list = ['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                            'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
                            'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
                            'WoodDeckSF', 'OpenPorchSF']
        elif mode == Filter_type.MID:
            feature_list = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF',
                            '1stFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces',
                            'GarageYrBlt', 'GarageCars']

        elif mode == Filter_type.LOW:
            feature_list = ['OverallQual',
                            'TotalBsmtSF', 'GrLivArea', 'GarageCars']

        df_selected = df[feature_list]
        if self.dataset_type == Dataset_type.TRAIN:
            df_label = df["SalePrice"]
            numpy_label = df_label.values
        numpy_data = df_selected.values

        mean = np.nanmean(numpy_data, axis=0)
        std = np.nanstd(numpy_data, axis=0)

        # 정규화 수행
        normalized_array = (((numpy_data - mean) / (std) * nor_std)) + nor_mean
        normalized_array = np.nan_to_num(normalized_array)
        if self.dataset_type == Dataset_type.TRAIN:
            return (normalized_array, numpy_label)
        return normalized_array


In [329]:
train_set = House_dataset(
    df_path=train_path, dataset_type=Dataset_type.TRAIN, mode=Filter_type.HIGH, nor_mean=0.5)

In [262]:
train_set[0]


0.8472732197365056


(array([0.79196567, 1.65147924, 2.05099379, 1.87866809, 1.51001534,
        1.57542484, 0.54069746, 0.20656621, 2.16185159, 1.37033344,
        1.78974052, 1.91220977, 0.04877351, 1.99242589, 1.31172464,
        0.24782416, 1.21650316]),
 0.8472732197365056)

In [109]:
train_set[0]


(array([0.79196567, 1.65147924, 2.05099379, 1.87866809, 1.51001534,
        1.57542484, 0.54069746, 0.20656621, 2.16185159, 1.37033344,
        1.78974052, 1.91220977, 0.04877351, 1.99242589, 1.31172464,
        0.24782416, 1.21650316]),
 208500)

In [115]:
# device = torch.device("mps")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [336]:
class Linear(nn.Module):
    def __init__(self, input_dim):
        super(Linear, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):

        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = nn.ReLU()(self.fc3(x))
        x = self.fc4(x)
        return x


In [333]:
import sys


def train(dataset, num_of_epoch, valid_rate, batch_size,
          optimizer, scheduler, criterion, valid_random_seed, patience=3):
    torch.manual_seed(valid_random_seed)

    optimizer = optimizer
    criterion = criterion

    train_set_count = int(len(dataset) * valid_rate)
    val_set_count = len(dataset) - train_set_count

    train_set, val_set = random_split(
        dataset, [train_set_count, val_set_count])

    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size,
                            shuffle=True)
    scheduler = scheduler

    train_losses = []
    val_losses = []
    lrs = []
    best_val_loss = sys.maxsize
    current_patience = patience
    is_early_stopping = False

    for epoch in range(num_of_epoch):
        model.train()
        correct_train = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            # print(data.shape)
            # print(data.dtype)
            data = data.type(torch.float32)
            output = model(data).float()
            target = target.float()
            loss = torch.sqrt(criterion(output, target)).float()
            loss.backward()
            optimizer.step()
            lrs.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
            # _, pred = torch.max(output.data, 1)
            # preds.extend(pred.tolist())
            # targets.extend(target.tolist())

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, lrs: {}'.format(
                    epoch + 1, batch_idx *
                    len(data[0]), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item(), lrs[-1]))

        train_losses.append(loss.item())
        print(
            f"Epoch: {epoch + 1} - train loss: {train_losses[-1]}")

        gc.collect()

        model.eval()
        preds = []
        targets = []
        correct_val = 0

        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                data = data.to(device)
                data = data.type(torch.float32)
                output = model(data).float()
                target = target.float()
                loss = torch.sqrt(criterion(output, target)).float()
                # _, pred = torch.max(output.data, 1)
                # preds.extend(pred.tolist())
                # targets.extend(target.tolist())
                # gc.collect()

            val_loss = loss.item()
            val_losses.append(val_loss)

            if val_loss < best_val_loss:
                current_patience = patience
                best_val_loss = val_loss
                best_model = model.state_dict()
                torch.save(best_model, 'model.pt')
            else:
                current_patience -= 1
                if current_patience < 0:
                    # print("Early Stopping!")
                    is_early_stopping = False

            print('Epoch {} finished: train loss = {}, val loss = {}'.format(epoch + 1,
                                                                             train_losses[-1], val_losses[-1]))
            print(
                f"Epoch: {epoch + 1} - Validation loss: {val_loss}")
            print("\n\n\n")

            if is_early_stopping:
                break

In [334]:
input_dim = len(train_set[0][0])
print(input_dim)
model = Linear(input_dim=input_dim)


17


In [332]:
model.load_state_dict(torch.load('model.pt'))


FileNotFoundError: [Errno 2] No such file or directory: 'model.pt'

In [335]:
valid_rate = 0.9
batch_size = 64
learning_rate = 0.0005
num_of_epoch = 100
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingWarmupRestarts(
    optimizer, first_cycle_steps=4000, cycle_mult=1.0, max_lr=0.001, min_lr=0.0001, warmup_steps=500, gamma=1)

criterion = nn.MSELoss()


valid_random_seed = 5
train(dataset=train_set, valid_rate=valid_rate, num_of_epoch=num_of_epoch, batch_size=64, optimizer=optimizer,
      scheduler=scheduler, criterion=criterion, valid_random_seed=valid_random_seed)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1 and 17x1)

In [310]:
def test(dataset, batch_size, trg_mean, trg_std, nor_mean, nor_std):

    test_loader = DataLoader(dataset, batch_size=batch_size,
                             shuffle=False)
    model.eval()
    pred_list = []

    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            data = data.to(device)
            data = data.type(torch.float32)
            output = model(data).float()
            output = trg_std * nor_std * (output - nor_mean) + trg_mean

            pred_list.extend(list(output))
    return pred_list

In [323]:
test_set = House_dataset(
    df_path=test_path, dataset_type=Dataset_type.TEST, mode=Filter_type.LOW, nor_mean=0.5)

result = test(dataset=test_set, batch_size=batch_size, trg_mean=train_set.trg_mean,
              trg_std=train_set.trg_std, nor_mean=train_set.nor_mean, nor_std=train_set.nor_std)

# result


In [324]:
submission_df.head(5)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [327]:
submission_df["SalePrice"] = [element.item() for element in result]
submission_df.head()

,Id,SalePrice
0,1461,173835.71875
1,1462,176700.53125
2,1463,181659.53125
3,1464,179474.75000
4,1465,175534.34375


In [328]:
submission_df.to_csv("submission_result.csv", index=False)